<a href="https://colab.research.google.com/github/atstuyuki/mediapipe/blob/main/mediapipe_for_movie__Pose__20220204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [1]:
# ライブラリーのインストール
!pip install numpy==1.19.3
!pip install mediapipe
#PySimpleGUIは後々組み込む
!pip install PySimpleGUI
# github からコードをコピー
#!git clone https://github.com/cedro3/mediapipe.git
#%cd mediapipe/

     |████████████████████████████████| 14.9 MB 4.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 32.7 MB 1.5 MB/s 
     |████████████████████████████████| 396 kB 4.5 MB/s 


In [2]:
# 初期設定 holistic とPoseを用意する　検出感度は0.5＞＞変更可能
import mediapipe as mp
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
# Initialize MediaPipe Holistic.
holistic = mp_holistic.Holistic(
    static_image_mode=True, min_detection_confidence=0.6)
pose= mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.6)
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(color=[225,0,200],thickness=2, circle_radius=1) #color=[B,G,R]で指定,線の太さと〇の色も指定可能

In [3]:
#3次元座標の角度計算の定義
def calculate_3D_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #0がx座標　１がｙ座標　２がｚ座標
    #ベクトルを定義 長さと内積からコサインを計算
    vec_a=a-b
    vec_c=c-b
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_c=np.linalg.norm(vec_c)
    inner_product=np.inner(vec_a, vec_c)
    cos=inner_product/(len_vec_a*len_vec_c)
    
    #角度（ラジアン）の計算
    rad=np.arccos(cos)
    
    #ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

In [4]:
#2次元からの角度計算の定義
def calculate_2D_angle(a,b,c):
  a=np.array(a)#firtst
  b=np.array(b)#second
  c=np.array(c)#third
  radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
  angle=np.abs(radians*180/np.pi)

  if angle>180.0:
    angle=360-angle
  
  return angle

In [5]:
#手元の動画をgoogle colabにupload

from google.colab import files
import os
uploaded = files.upload()
#uploadedは辞書型なのでファイル名を取り出し、絶対パスを取得してfullpathという変数を当てる
for k in uploaded.keys():
  print('file name:',k)
fullpath=os.path.abspath(k)
print('fullpath:',fullpath)

Saving throwing2.mp4 to throwing2.mp4
file name: throwing2.mp4
fullpath: /content/throwing2.mp4


# サンプルビデオの読み込み＋静止画に分解

In [6]:
# video2images
import os
import shutil
import cv2

# 既にimagesフォルダーがあれば削除
if os.path.isdir('images'):
    shutil.rmtree('images')

os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= fullpath,   
                   image_dir='./images/', 
                   image_file='%s.png'):
 
    # Initial setting
    i = 0
    interval = 3#適宜調整
    length = 300
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
 
def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

In [7]:
#imageフォルダの最初の1枚目の画像を取得
firstimage =cv2.imread('./images/000000.png')
#imageのwidthとheightを取得
height, width, channels = firstimage.shape[:3]

In [8]:
#pose_landmarkのリストを取得して配列に変換
import numpy as np
landmarklist=[]#空の配列を作成
for index in range(len(mp_holistic.PoseLandmark)):
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_x')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_y')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_z')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_visibility')
#リストをarrayに変換
landmarklist=np.array(landmarklist).reshape(1,-1)
#landmarkshape の内容の確認
print(landmarklist.dtype)
print(landmarklist)
print(landmarklist.shape)

<U27
[['NOSE_x' 'NOSE_y' 'NOSE_z' 'NOSE_visibility' 'LEFT_EYE_INNER_x'
  'LEFT_EYE_INNER_y' 'LEFT_EYE_INNER_z' 'LEFT_EYE_INNER_visibility'
  'LEFT_EYE_x' 'LEFT_EYE_y' 'LEFT_EYE_z' 'LEFT_EYE_visibility'
  'LEFT_EYE_OUTER_x' 'LEFT_EYE_OUTER_y' 'LEFT_EYE_OUTER_z'
  'LEFT_EYE_OUTER_visibility' 'RIGHT_EYE_INNER_x' 'RIGHT_EYE_INNER_y'
  'RIGHT_EYE_INNER_z' 'RIGHT_EYE_INNER_visibility' 'RIGHT_EYE_x'
  'RIGHT_EYE_y' 'RIGHT_EYE_z' 'RIGHT_EYE_visibility' 'RIGHT_EYE_OUTER_x'
  'RIGHT_EYE_OUTER_y' 'RIGHT_EYE_OUTER_z' 'RIGHT_EYE_OUTER_visibility'
  'LEFT_EAR_x' 'LEFT_EAR_y' 'LEFT_EAR_z' 'LEFT_EAR_visibility'
  'RIGHT_EAR_x' 'RIGHT_EAR_y' 'RIGHT_EAR_z' 'RIGHT_EAR_visibility'
  'MOUTH_LEFT_x' 'MOUTH_LEFT_y' 'MOUTH_LEFT_z' 'MOUTH_LEFT_visibility'
  'MOUTH_RIGHT_x' 'MOUTH_RIGHT_y' 'MOUTH_RIGHT_z'
  'MOUTH_RIGHT_visibility' 'LEFT_SHOULDER_x' 'LEFT_SHOULDER_y'
  'LEFT_SHOULDER_z' 'LEFT_SHOULDER_visibility' 'RIGHT_SHOULDER_x'
  'RIGHT_SHOULDER_y' 'RIGHT_SHOULDER_z' 'RIGHT_SHOULDER_visibility'
  'LEFT_ELBO

# MediaPipeで静止画を処理

In [43]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob
import time
#変数の初期化
rtelbowAnglelist=[]
rtshoulderAnglelist=[]
timelist=[]
data_land = np.zeros((0,132))

# image file names to files in list format
files=[]
for name in sorted(glob.glob('./images/*.png')):
    files.append(name)

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in files}
w = width             # 画像の横幅を取得
h = height            # 画像の縦幅を取得

for name, image in images.items():
  # Convert the BGR image to RGB and process it with MediaPipe Pose.
  if image is None:
      break
  results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  landmarks = results.pose_landmarks
  #get coordinate poseの33か所の3次元座標データと確信度(visibilisty)をすべて結合してCSVに変換
  data_land2 = np.zeros((1,4))

  if landmarks:
    for x in range (0,32):
        data1= results.pose_landmarks.landmark[x].x
        data2 = results.pose_landmarks.landmark[x].y
        data3 = results.pose_landmarks.landmark[x].z
        data4 = results.pose_landmarks.landmark[x].visibility
        keydata = np.hstack((data1,data2,data3,data4)).reshape(1,-1)
        data_land2 = np.hstack((data_land2,keydata))
  else:
        #keydata = np.zeros(132)
        data_land2 = np.zeros(136)
        
  data_land2 = data_land2[4:]
  data_land = np.vstack((data_land,data_land2))
  final_data= np.vstack((landmarklist,data_land))

  if landmarks is None:#Noneの時にnanを入れる処理をここに入れる

      continue
#各検出ポイントをarrayに変換
  rtshoulder=[results.pose_landmarks.landmark[12].x,results.pose_landmarks.landmark[12].y,results.pose_landmarks.landmark[12].z]
  rtelbow=[results.pose_landmarks.landmark[14].x,results.pose_landmarks.landmark[14].y,results.pose_landmarks.landmark[14].z]
  rtwrist=[results.pose_landmarks.landmark[16].x,results.pose_landmarks.landmark[16].y,results.pose_landmarks.landmark[16].z]
  rthip=[results.pose_landmarks.landmark[24].x,results.pose_landmarks.landmark[24].y,results.pose_landmarks.landmark[24].z]
  ltshoulder=[results.pose_landmarks.landmark[11].x,results.pose_landmarks.landmark[11].y,results.pose_landmarks.landmark[11].z]
  #3D座標から角度を計算 2Dか3Dか選ぶ
  rtelbowAngle=180-calculate_2D_angle(rtshoulder,rtelbow,rtwrist)
  rtshoulderAngle=calculate_2D_angle(rthip,rtshoulder,rtelbow)#肘下がり検出用に今回のshoulderAngleは左肩-右肩-右肘のなす角
  #角度情報をテキストして画像に挿入
  cv2.putText(image, 'rtElbow'+str(round(rtelbowAngle,1)),
              (200,400),
              #tuple(np.multiply(rtelbow[:2],[w, h]).astype(int)),
              cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,50),2,cv2.LINE_AA)

  cv2.putText(image, 'rtshoulder'+str(round(rtshoulderAngle,1)),
              (200,350),
              #tuple(np.multiply(rtshoulder[:2],[w, h]).astype(int)),
              cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,50),2,cv2.LINE_AA)

        
  rtelbowAnglelist.append(rtelbowAngle)
  rtshoulderAnglelist.append(rtshoulderAngle)  

  # Draw pose landmarks.
  annotated_image = image.copy()
  # Get Landmarks
  #landmarks=results.pose_landmarks.landmark
 
  """
  Holisticを使用しない場合はこの部分をコメントアウト
  
  mp_drawing.draw_landmarks(annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.face_landmarks, 
      connections=mp_holistic.FACEMESH_TESSELATION,
      landmark_drawing_spec=drawing_spec,
      connection_drawing_spec=drawing_spec)
  """
  mp_drawing.draw_landmarks(
        image=annotated_image, 
        landmark_list=results.pose_landmarks, 
        connections=mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=drawing_spec,
        connection_drawing_spec=drawing_spec)
  #get coordinate poseの33か所の3次元座標データと確信度(visibilisty)をすべて結合してCSVに変換
  """data_land2 = np.zeros((1,3))
    
  for x in range (0,33):
      data1 = results.pose_landmarks.landmark[x].x
      data2 = results.pose_landmarks.landmark[x].y
      data3 = results.pose_landmarks.landmark[x].z
      data4 = results.pose_landmarks.landmark[x].visibility
      keydata = np.hstack((data1,data2,data3,data4)).reshape(1,-1)
      data_land2 = np.hstack((data_land2,keydata))

  data_land2 = data_land2[:,3:]
  data_land = np.vstack((data_land,data_land2))
  final_data= np.vstack((landmarklist,data_land))"""
  cv2.imwrite(name, annotated_image)  
#savetxtではデータの型が合わないと保存的無いことがあるのでfmtで文字の型を指定
  np.savetxt('keypoint_results.csv',final_data,delimiter = ',',fmt = '%s')


In [42]:
keydata

array([[0.53980517, 0.96170902, 0.06307624, 0.92065257]])

In [40]:
print(data_land.shape)
print(data_land2.shape)

(1, 132)
(0, 132)


In [45]:
landmarks

landmark {
  x: 0.6194627285003662
  y: 0.3779260516166687
  z: -0.04166622459888458
  visibility: 0.9999370574951172
}
landmark {
  x: 0.618613064289093
  y: 0.357524573802948
  z: -0.030064543709158897
  visibility: 0.9998655319213867
}
landmark {
  x: 0.6183863878250122
  y: 0.35653918981552124
  z: -0.030162019655108452
  visibility: 0.9999074935913086
}
landmark {
  x: 0.6179524660110474
  y: 0.35549309849739075
  z: -0.03023620881140232
  visibility: 0.9999077320098877
}
landmark {
  x: 0.6154065132141113
  y: 0.358303040266037
  z: -0.06567729264497757
  visibility: 0.9998780488967896
}
landmark {
  x: 0.6129834055900574
  y: 0.35804736614227295
  z: -0.0657399445772171
  visibility: 0.9998862743377686
}
landmark {
  x: 0.6101657152175903
  y: 0.3575899004936218
  z: -0.06581495702266693
  visibility: 0.9998748302459717
}
landmark {
  x: 0.604850172996521
  y: 0.35567769408226013
  z: 0.037641964852809906
  visibility: 0.9998825788497925
}
landmark {
  x: 0.5953882336616516
  y:

In [39]:
#CSVデータと画像の枚数が一致しているか確認
print(final_data.shape)
print(len(images))

(105, 132)
108


# 処理した画像をmp4動画に変換


In [ ]:
# 既に output.mp4 があれば削除
import os
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

! ffmpeg -r 10 -i images/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
# --- mp4動画の再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="85%" height="85%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

＊作成したmp4動画は、左側のファイル欄にある　./mediapipe/output.mp4 を右クリックするとダウンロードできます。

# 処理した画像をgif動画に変換


In [ ]:
# images2gif
from PIL import Image
import glob
 
files = sorted(glob.glob('./images/*.png'))
images = list(map(lambda file: Image.open(file), files))
images[0].save('./out.gif', save_all=True, 
               append_images=images[1:], 
               duration=100, loop=0)

In [ ]:
# サイズの大きいgifは再生しないで下さい（クラッシュするため）。
# display gif
from IPython.display import Image
Image('./out.gif', format='png')

＊作成したgif動画は、左側のファイル欄にある ./mediapipe/out.gif を右クリックするとダウンロードできます。